<h1> Hyperparameter tuning </h1>

This notebook is Lab4b of CPB 102, Google's course on Machine Learning using Cloud ML.

This notebook builds on Lab 4a, adding hyperparameter tuning to the feature engineering done in that lab.  To save time, we will start from the preprocessed data in that lab.

<div id="toc"></div>

In [4]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [28]:
%bash
# remember to "Reset Session" if you execute this cell -- this is needed to restart the Python kernel with updated package
#gsutil cp gs://cloud-ml/sdk/cloudml-0.1.4.tar.gz .
pip install --force-reinstall --upgrade cloudml-0.1.4.tar.gz

Processing ./cloudml-0.1.4.tar.gz
  Using cached six-1.10.0-py2.py3-none-any.whl
  Using cached numpy-1.11.1-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached Pillow-3.3.1-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached dpkt-1.8.8-py2-none-any.whl
  Using cached rsa-3.4.2-py2.py3-none-any.whl
  Using cached pyasn1-0.1.9-py2.py3-none-any.whl
  Using cached pyasn1_modules-0.0.8-py2.py3-none-any.whl
  Using cached google_apitools-0.5.4-py2-none-any.whl
  Using cached protorpc-0.11.1-py2-none-any.whl
  Using cached mock-2.0.0-py2.py3-none-any.whl
  Using cached beautifulsoup4-4.5.1-py2-none-any.whl
  Using cached setuptools-27.2.0-py2.py3-none-any.whl
  Using cached funcsigs-1.0.2-py2.py3-none-any.whl
  Using cached pbr-1.10.0-py2.py3-none-any.whl
  Running setup.py bdist_wheel for cloudml: started
  Running setup.py bdist_wheel for cloudml: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/24/e7/b6/341a6eb8b4b636be9f27d6c6de774d0227faae264846ff02f3
Successfully bu

<h1> Retreiving preprocessed data </h1>

To save time, we'll go off the preprocessed data from Lab4a. Since we don't know whether you got Lab4a working completely, let's start off from my Lab4a results.  Change the BUCKET below to be yours.

In [11]:
%bash
BUCKET=cloud-training-demos
SOURCE=gs://cloud-training-demos/taxifare/taxi_preproc4a_saved/*
gsutil cp $SOURCE gs://$BUCKET/taxifare/taxi_preproc4b

Copying gs://cloud-training-demos/taxifare/taxi_preproc4a_saved/features_eval [Content-Type=application/x-www-form-urlencoded]...
Copying     ...ning-demos/taxifare/taxi_preproc4b/features_eval: 293.47 KiB/293.47 KiB    
Copying gs://cloud-training-demos/taxifare/taxi_preproc4a_saved/features_train [Content-Type=application/x-www-form-urlencoded]...
Copying     ...ing-demos/taxifare/taxi_preproc4b/features_train: 328.52 KiB/328.52 KiB    
Copying gs://cloud-training-demos/taxifare/taxi_preproc4a_saved/info [Content-Type=text/plain]...
Copying     ...loud-training-demos/taxifare/taxi_preproc4b/info: 461 B/461 B    
Copying gs://cloud-training-demos/taxifare/taxi_preproc4a_saved/metadata.yaml [Content-Type=text/plain]...
Copying     ...ning-demos/taxifare/taxi_preproc4b/metadata.yaml: 2.25 KiB/2.25 KiB    


<h2> Modify TensorFlow code: change optimizable values to command-line arguments </h2>

In [12]:
%bash
grep -3 hyperparam taxifare/trainer/*.py

taxifare/trainer/task.py-      _, train_examples = taxifare.read_examples(
taxifare/trainer/task.py-          training_data, HYPERPARAMS['batch_size'], shuffle=False)
taxifare/trainer/task.py-
taxifare/trainer/task.py:      # hyperparams
taxifare/trainer/task.py:      hyperparams = {
taxifare/trainer/task.py-        'hidden_layer1_size': args.hidden1,
taxifare/trainer/task.py-        'hidden_layer2_size': args.hidden2,
taxifare/trainer/task.py-        'hidden_layer3_size': args.hidden3,
--
taxifare/trainer/task.py-
taxifare/trainer/task.py-      # Generate placeholders for the examples.
taxifare/trainer/task.py-      placeholder, inputs, targets, _ = (
taxifare/trainer/task.py:          taxifare.create_inputs(metadata, input_data=train_examples, hyperparams))
taxifare/trainer/task.py-
taxifare/trainer/task.py-      # Build a Graph that computes predictions from the inference model.
taxifare/trainer/task.py:      output = taxifare.inference(inputs, metadata, hyperparams)
taxifare/traine

<h2> Train once </h2>

Here, we package up the code and train as normal.

In [28]:
%bash
rm -rf taxifare.tar.gz taxi_trained
tar cvfz taxifare.tar.gz taxifare
gsutil cp taxifare.tar.gz gs://cloud-training-demos/taxifare/source4b/taxifare.tar.gz

taxifare/
taxifare/PKG-INFO
taxifare/setup.cfg
taxifare/setup.py
taxifare/trainer/
taxifare/trainer/__init__.py
taxifare/trainer/task.py
taxifare/trainer/taxifare.py
taxifare/trainer.egg-info/
taxifare/trainer.egg-info/dependency_links.txt
taxifare/trainer.egg-info/PKG-INFO
taxifare/trainer.egg-info/SOURCES.txt
taxifare/trainer.egg-info/top_level.txt


Copying file://taxifare.tar.gz [Content-Type=application/x-tar]...
Uploading   ...training-demos/taxifare/source4b/taxifare.tar.gz: 7.42 KiB/7.42 KiB    


In [18]:
%bash
gsutil cp -R gs://cloud-training-demos/taxifare/taxi_preproc4b /content/CPB102/lab4b

Copying gs://cloud-training-demos/taxifare/taxi_preproc4b/features_eval...
Copying gs://cloud-training-demos/taxifare/taxi_preproc4b/features_train...
Copying gs://cloud-training-demos/taxifare/taxi_preproc4b/info...
Copying gs://cloud-training-demos/taxifare/taxi_preproc4b/metadata.yaml...


In [27]:
%%ml train
package_uris: /content/CPB102/lab4b/taxifare.tar.gz
python_module: trainer.task
scale_tier: BASIC
region: us-central1
args:
  train_data_paths:
    - /content/CPB102/lab4b/taxi_preproc4b/features_train
  eval_data_paths:
    - /content/CPB102/lab4b/taxi_preproc4b/features_eval
  metadata_path: /content/CPB102/lab4b/taxi_preproc4b/metadata.yaml
  output_path: /content/CPB102/lab4b/taxi_trained
  max_steps: 500
  hidden1:  64
  hidden2:   8
  hidden3:   4
  nbuckets:  5

Job Running...

master log

master: E tensorflow/core/client/tensor_c_api.cc:485] Enqueue operation was cancelled master: [[Node: batch/fifo_queue_enqueue = QueueEnqueue[Tcomponents=[DT_STRING, DT_STRING], _class=["loc:@batch/fifo_queue"], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](batch/fifo_queue, ReaderRead, ReaderRead:1)]] master: E tensorflow/core/client/tensor_c_api.cc:485] Enqueue operation was cancelled master: [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueMany[Tcomponents=[DT_STRING], _class=["loc:@input_producer"], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/Identity)]] master: Final error after 500 steps = 8.676 master: E tensorflow/core/client/tensor_c_api.cc:485] Enqueue operation was cancelled master: [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueMany[Tcomponents=[DT_STRING], _class=["loc:@input_producer"], timeout_ms=-1, _device="/job:master/replica:0/task:0/cpu:0"](input_producer, input_producer/Identity)]] master: E tensorflow/core/client/tensor_c_api.cc:485] Enqueue operation was cancelled master: [[Node: batch/fifo_queue_enqueue = QueueEnqueue[Tcomponents=[DT_STRING, DT_STRING], _class=["loc:@batch/fifo_queue"], timeout_ms=-1, _device="/job:master/replica:0/task:0/cpu:0"](batch/fifo_queue, ReaderRead, ReaderRead:1)]] master: plat = [None, 1] master: latdist = [None, 1] master: pickup = None master: pickupdropoff = None master: inputs= [None, None] master: Done training. master: E tensorflow/core/client/tensor_c_api.cc:485] Enqueue operation was cancelled master: [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueMany[Tcomponents=[DT_STRING], _class=["loc:@input_producer"], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer, input_producer/Identity)]] master: E tensorflow/core/client/tensor_c_api.cc:485] Enqueue operation was cancelled master: [[Node: batch/fifo_queue_enqueue = QueueEnqueue[Tcomponents=[DT_STRING, DT_STRING], _class=["loc:@batch/fifo_queue"], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](batch/fifo_queue, ReaderRead, ReaderRead:1)]] master:

Job Finished.

<h2> Hyperparameter training </h2>

Now, we carry out the training, but this time on the cloud, and this time with some hyperparameters

In [29]:
%%ml train --cloud
package_uris: gs://cloud-training-demos/taxifare/source4b/taxifare.tar.gz
python_module: trainer.task
scale_tier: BASIC
region: us-central1
args:
  train_data_paths:
    - gs://cloud-training-demos/taxifare/taxi_preproc4b/features_train
  eval_data_paths:
    - gs://cloud-training-demos/taxifare/taxi_preproc4b/features_eval
  metadata_path: gs://cloud-training-demos/taxifare/taxi_preproc4b/metadata.yaml
  output_path: gs://cloud-training-demos/taxifare/taxi_trained4b
  max_steps: 500
  hidden1:  64
  hidden2:   8
  hidden3:   4
  nbuckets:  5

In [30]:
%ml train --cloud

Parameters,Local Run Required,Cloud Run Required,Description
package_uris,True,True,A GCS or local (for local run only) path to your python training program package.
python_module,True,True,The module to run.
scale_tier,False,True,"Type of resources requested for the job. On local run, BASIC means 1 master process only, and any other values mean 1 master 1 worker and 1 ps processes. But you can also override the values by setting worker_count and parameter_server_count. On cloud, see service definition for possible values."
region,False,True,Where the training job runs. For cloud run only.
args,False,False,Args that will be passed to your training program.


In [ ]:
%%ml train [--cloud]
package_uris: gs://your-bucket/my-training-package.tar.gz
python_module: your_program.your_module
scale_tier: BASIC
region: us-central1
args:
  string_arg: value
  int_arg: value
  appendable_arg:
    - value1
    - value2


In [7]:
!gsutil ls gs://cloud-training-demos/taxifare/taxi_trained4b

ls: cannot access /content/CPB102/lab2d/taxi_trained: No such file or directory


In [8]:
%tensorboard start --logdir gs://cloud-training-demos/taxifare/taxi_trained4b

In [9]:
%tensorboard stop --pid 3222

In [20]:
%ml summary --dir gs://cloud-training-demos/taxifare/taxi_trained4b/summaries  gs://cloud-training-demos/taxifare/taxi_trained4b/eval  --name loss error --step

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License